In [4]:
from datetime import date, datetime, timedelta

import requests
import pandas as pd
import numpy as np
from pprint import pprint
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter

%matplotlib inline
sns.set_palette('muted')
sns.set_style('white')

In [5]:
# write your solution here
def geocode_location(city, country):
    url = 'https://geocode.xyz/'
    api_key = '108980271663949389508x34567'
    json=1
    req_url = f"{url},{city},{country}?json={json}&auth={api_key}"
    r = requests.get(req_url)
    result = r.json()
    coord = (float(result['longt']), float(result['latt']))
    return coord
geocode_location("london", "united kingdom")

(-0.11485, 51.51417)

In [ ]:
# helper function for 15-day date range
def get_date_range(birthday):
    return {'start_date': birthday - timedelta(days=7),
           'end_date':birthday + timedelta(days=7)}

In [ ]:
 def get_api_response(latitude,longitude,birthday):
    url = "https://archive-api.open-meteo.com/v1/archive"    
    date_range = get_date_range(birthday)
    print(f"start range:{date_range['start_date']}, end_date:{date_range['end_date']}")    
    params = {'latitude':latitude,
             'longitude':longitude,
             'start_date':date_range['start_date'],
             'end_date': date_range['end_date'],
             'daily':['temperature_2m_max','temperature_2m_min','apparent_temperature_max','apparent_temperature_min','precipitation_sum'],
              'timezone':'auto',
              'temperature_unit':'fahrenheit',
              'precipitation_unit':'inch'
             }
    r = requests.get(url, params=params)
    response = r.json()
    return response

In [ ]:
def get_all_response_objects(latitude, longitude, birthday):
    responses = []    
    for year in range(1960,2023):
        request_date = date(year, birthday.month, birthday.day)
        response = get_api_response(latitude,longitude, request_date)
        responses.append(response)
    return responses

In [ ]:
birthday = datetime.strptime('10-19-1980', '%m-%d-%Y').date()
print(birthday)
api_return = get_api_response('42.50496','-83.15053',birthday)
pprint(api_return)

In [ ]:
birthday = datetime.strptime('10-19-1991', '%m-%d-%Y').date()
print(birthday)
api_return_all = get_all_response_objects('42.50496','-83.15053',birthday)
pprint(api_return_all)


def get_data():
    birthday = get_user_birthday()
    location = get_user_city_of_birth()
    def get_all_response_objects(latitude, longitude, birthday):
        responses = []    
        for year in range(1960,2023):
            request_date = date(year, birthday.month, birthday.day)
            response = get_api_response(latitude,longitude, request_date)
            responses.append(response)
        return responses
    def get_api_response(latitude,longitude,birthday):
        url = "https://archive-api.open-meteo.com/v1/archive"    
        date_range = get_date_range(birthday)
        print(f"start range:{date_range['start_date']}, end_date:{date_range['end_date']}")    
        params = {'latitude':latitude,
                 'longitude':longitude,
                 'start_date':date_range['start_date'],
                 'end_date': date_range['end_date'],
                 'daily':['temperature_2m_max','temperature_2m_min','apparent_temperature_max','apparent_temperature_min','precipitation_sum'],
                  'timezone':'auto',
                  'temperature_unit':'fahrenheit',
                  'precipitation_unit':'inch'
                 }
        r = requests.get(url, params=params)
        response = r.json()
        return response
    def get_date_range(birthday):
        return {'start_date': birthday - timedelta(days=7),
               'end_date':birthday + timedelta(days=7)}
def get_user_birthday():
    birthday = input('Enter your birthday (MM/DD/YYYY): ')
    return datetime.strptime(birthday, "%m/%d/%Y")

def get_user_city_of_birth():
    city = input('Enter the city you were born in: ')
    country = input('Enter the country you were born in: ')
    return city , country
df = get_data()

df

In [ ]:
def parse_json_response(responses):
    df=pd.DataFrame(responses)
    df = df.join(pd.json_normalize(df['daily'])).drop(columns=['daily'])   
    display(df)
parse_json_response(api_return_all)

In [ ]:
def parse_json_response(responses):
    df = pd.DataFrame(responses)
    df = df.join(pd.json_normalize(df['daily'])).drop(columns=['daily'])
    measures = ["temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", "precipitation_sum"]      
    for col in measures:
         df[f"{col}_mean"] = df[col].apply(np.mean)
    #chop up time to add year column.
    df['year'] = df['time'].apply(lambda x: x[7][:4])
    df = df[['year','temperature_2m_max_mean','temperature_2m_min_mean','apparent_temperature_max_mean','apparent_temperature_min_mean','precipitation_sum_mean']]    
    return df    
    display(df)
df1 = parse_json_response(api_return_all)
df1

In [ ]:
import matplotlib.pyplot as plt

# Assuming df is the parsed DataFrame returned by the parse_json_response function

# Plotting temperature data
plt.figure(figsize=(10, 6))

# Plotting mean temperatures over the years
plt.plot(df1['year'], df1['temperature_2m_max_mean'], label='Max Temperature', color='red')
plt.plot(df1['year'], df1['temperature_2m_min_mean'], label='Min Temperature', color='blue')
plt.plot(df1['year'], df1['apparent_temperature_max_mean'], label='Max Apparent Temperature', color='orange')
plt.plot(df1['year'], df1['apparent_temperature_min_mean'], label='Min Apparent Temperature', color='green')

plt.xlabel('Year')
plt.ylabel('Temperature (°C)')
plt.title('Change of Temperature Over Years')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plotting precipitation data
plt.figure(figsize=(10, 6))

# Plotting mean precipitation over the years
plt.plot(df1['year'], df1['precipitation_sum_mean'], label='Mean Precipitation', color='blue')

plt.xlabel('Year')
plt.ylabel('Precipitation (mm)')
plt.title('Change of Precipitation Over Years')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


each variable has high and low outliers with the lowest point slightly rising each year but for the most part the temperature remains in expected range